In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

data = pd.read_csv("/content/ourdata.csv")
print(data.head())

         Date Time_Start  Time_End Crop_Type  Base_Hourly_Wage  \
0  2023-01-26   07:30:00  16:30:00     Wheat              12.0   
1  2023-01-24   07:30:00  15:30:00  Soybeans              12.5   
2  2023-01-20   07:30:00  16:30:00     Wheat              12.0   
3  2023-01-31   08:00:00  17:00:00      Corn              11.5   
4  2023-01-27   09:00:00  17:00:00      Corn              11.5   

   Supply_Demand_Ratio  Dynamic_Pricing_Multiplier  Total_Earnings  
0                  1.2                       1.495        161.4600  
1                  1.3                       1.625        162.5000  
2                  1.4                       1.495        161.4600  
3                  1.1                       1.375        142.3125  
4                  1.4                       1.625        149.5000  


In [6]:
print(data.describe())

       Base_Hourly_Wage  Supply_Demand_Ratio  Dynamic_Pricing_Multiplier  \
count       5000.000000          5000.000000                 5000.000000   
mean          12.001900             1.296340                    1.535514   
std            0.405684             0.141607                    0.116130   
min           11.500000             1.100000                    1.375000   
25%           11.500000             1.200000                    1.440000   
50%           12.000000             1.300000                    1.495000   
75%           12.500000             1.400000                    1.625000   
max           12.500000             1.500000                    1.725000   

       Total_Earnings  
count     5000.000000  
mean       156.467542  
std         15.978918  
min        126.500000  
25%        143.520000  
50%        155.520000  
75%        168.187500  
max        194.062500  


In [7]:

# Convert 'Time_Start' and 'Time_End' columns to datetime format
data['Time_Start'] = pd.to_datetime(data['Time_Start'])
data['Time_End'] = pd.to_datetime(data['Time_End'])

# Calculate duration in hours
data['Duration'] = (data['Time_End'] - data['Time_Start']).dt.total_seconds() / 3600

# Print the DataFrame with the added 'Duration' column
print(data)


            Date          Time_Start            Time_End Crop_Type  \
0     2023-01-26 2024-02-19 07:30:00 2024-02-19 16:30:00     Wheat   
1     2023-01-24 2024-02-19 07:30:00 2024-02-19 15:30:00  Soybeans   
2     2023-01-20 2024-02-19 07:30:00 2024-02-19 16:30:00     Wheat   
3     2023-01-31 2024-02-19 08:00:00 2024-02-19 17:00:00      Corn   
4     2023-01-27 2024-02-19 09:00:00 2024-02-19 17:00:00      Corn   
...          ...                 ...                 ...       ...   
4995  2023-01-25 2024-02-19 09:00:00 2024-02-19 17:00:00  Soybeans   
4996  2023-02-01 2024-02-19 08:00:00 2024-02-19 16:00:00      Corn   
4997  2023-01-24 2024-02-19 09:00:00 2024-02-19 17:00:00     Wheat   
4998  2023-01-26 2024-02-19 08:00:00 2024-02-19 16:00:00      Corn   
4999  2023-01-26 2024-02-19 08:00:00 2024-02-19 16:00:00     Wheat   

      Base_Hourly_Wage  Supply_Demand_Ratio  Dynamic_Pricing_Multiplier  \
0                 12.0                  1.2                       1.495   
1        

In [10]:
import numpy as np

# Define percentiles for high and low values
high_percentile = 75
low_percentile = 25

# Calculate demand_multiplier based on percentile for high and low demand (using Total_Earnings as a proxy for demand)
data['demand_multiplier'] = np.where(data['Total_Earnings'] > np.percentile(data['Total_Earnings'], high_percentile),
                                     data['Total_Earnings'] / np.percentile(data['Total_Earnings'], high_percentile),
                                     data['Total_Earnings'] / np.percentile(data['Total_Earnings'], low_percentile))

# Calculate supply_multiplier based on percentile for high and low supply (using Base_Hourly_Wage * Supply_Demand_Ratio as a proxy for supply)
data['supply_multiplier'] = np.where((data['Base_Hourly_Wage'] * data['Supply_Demand_Ratio']) > np.percentile(data['Base_Hourly_Wage'] * data['Supply_Demand_Ratio'], low_percentile),
                                     np.percentile(data['Base_Hourly_Wage'] * data['Supply_Demand_Ratio'], high_percentile) / (data['Base_Hourly_Wage'] * data['Supply_Demand_Ratio']),
                                     np.percentile(data['Base_Hourly_Wage'] * data['Supply_Demand_Ratio'], low_percentile) / (data['Base_Hourly_Wage'] * data['Supply_Demand_Ratio']))

# Define thresholds for high and low demand/supply
demand_threshold_high = 1.2  # Higher demand threshold
demand_threshold_low = 0.8   # Lower demand threshold
supply_threshold_high = 0.8  # Higher supply threshold
supply_threshold_low = 1.2   # Lower supply threshold

# Calculate dynamic pricing multiplier
data['Dynamic_Pricing_Multiplier'] = (np.maximum(data['demand_multiplier'], demand_threshold_low) *
                                       np.maximum(data['supply_multiplier'], supply_threshold_high))

# Display the resulting DataFrame with the added 'Dynamic_Pricing_Multiplier' column
print(data)


            Date          Time_Start            Time_End Crop_Type  \
0     2023-01-26 2024-02-19 07:30:00 2024-02-19 16:30:00     Wheat   
1     2023-01-24 2024-02-19 07:30:00 2024-02-19 15:30:00  Soybeans   
2     2023-01-20 2024-02-19 07:30:00 2024-02-19 16:30:00     Wheat   
3     2023-01-31 2024-02-19 08:00:00 2024-02-19 17:00:00      Corn   
4     2023-01-27 2024-02-19 09:00:00 2024-02-19 17:00:00      Corn   
...          ...                 ...                 ...       ...   
4995  2023-01-25 2024-02-19 09:00:00 2024-02-19 17:00:00  Soybeans   
4996  2023-02-01 2024-02-19 08:00:00 2024-02-19 16:00:00      Corn   
4997  2023-01-24 2024-02-19 09:00:00 2024-02-19 17:00:00     Wheat   
4998  2023-01-26 2024-02-19 08:00:00 2024-02-19 16:00:00      Corn   
4999  2023-01-26 2024-02-19 08:00:00 2024-02-19 16:00:00     Wheat   

      Base_Hourly_Wage  Supply_Demand_Ratio  Dynamic_Pricing_Multiplier  \
0                 12.0                  1.2                    1.347656   
1        